In [30]:
import pandas as pd
import plotly.express as px
from IPython.display import display  # just needed for Jupyter Notebook display!

filepath = 'uploads/processed.csv'
df = pd.read_csv(filepath)
df.head(15)

,date,description,amount,auto_category,bank_category,transaction_type,needs_review
0,2025-01-31,WHOLEFDS TMC 10207,-27.18,Groceries,Groceries,Expense,False
1,2025-01-31,SQ *CORVUS COFFEE ROASTER,-12.08,Eating Out,Food & Drink,Expense,False
2,2025-01-30,PUDGE BROS PIZZA - DTC,-37.00,Eating Out,Food & Drink,Expense,False
3,2025-01-30,TARGET.COM *,-38.26,Shopping,Shopping,Expense,False
4,2025-01-29,AMAZON MKTPL*ZC1KD3O91,-46.12,Amazon,Shopping,Expense,True
5,2025-01-27,Stampede,-10.00,Food & Drink,Food & Drink,Expense,False
6,2025-01-26,TRADER JOE S #303,-107.56,Groceries,Groceries,Expense,False
7,2025-01-26,TST*URBAN EGG - BELLEVIE,-43.00,Eating Out,Food & Drink,Expense,False
8,2025-01-28,AMAZON MKTPL*Z77U43EC2,-28.65,Amazon,Shopping,Expense,True
9,2025-01-27,AMAZON MKTPL*ZC3N47T80,-55.73,Amazon,Shopping,Expense,True


In [31]:
# Ensure required columns exist
# CH wondering why this could be an issue give that tou create the file???
required_cols = ['date', 'amount', 'auto_category']
missing = [col for col in required_cols if col not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {', '.join(missing)}")
else:
    print("All required columns are present.")

All required columns are present.


In [32]:
# in auto_category replace missing values with 'Uncategorized'
df['auto_category'] = df['auto_category'].fillna('Uncategorized')   


In [33]:
# Group by auto_category and sum the absolute value of amount
df_pie = df.copy()
df_pie['amount'] = df_pie['amount'].abs() # so that spend amounts are positive for pie chart
pie_data = df_pie.groupby('auto_category', as_index=False)['amount'].sum()


charts = {}
charts['pie'] = px.pie(
    pie_data,
    values='amount',
    names='auto_category',
    title='Spending by Category (Percentage of Total)'
)
charts['pie'].update_traces(textinfo='percent+label')  # Show percent and label on slices
display(charts['pie'])


In [34]:
# Group by auto_category and sum the absolute value of amount for Expenses
df_expense = df[df['transaction_type'] == 'Expense'].copy()
df_expense['amount'] = df_expense['amount'].abs()
pie_data_expense = df_expense.groupby('auto_category', as_index=False)['amount'].sum()

# Group by auto_category and sum the absolute value of amount for Income
df_income = df[df['transaction_type'] == 'Income'].copy()
df_income['amount'] = df_income['amount'].abs()
pie_data_income = df_income.groupby('auto_category', as_index=False)['amount'].sum()

In [35]:
charts['pie_exp'] = px.pie(
    pie_data_expense,
    values='amount',
    names='auto_category',
    title='Expenses by Category (Percentage of Total)'
)
charts['pie_exp'].update_traces(textinfo='percent+label')  # Show percent and label on slices
display(charts['pie_exp'])

In [36]:
charts['pie_inc'] = px.pie(
    pie_data_income,
    values='amount',
    names='auto_category',
    title='Income by Category (Percentage of Total)'
)
charts['pie_inc'].update_traces(textinfo='percent+label')  # Show percent and label on slices
display(charts['pie_inc'])

In [37]:

if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])  # Ensure datetime

    # only use expenses!
    df_expense = df[df['transaction_type'] == 'Expense'].copy()
    df_expense['amount'] = df_expense['amount'].abs()

    # Group by date and sum amounts (daily spending)
    df_daily = df_expense.groupby('date', as_index=False)['amount'].sum()

    # Sort by date
    df_daily = df_daily.sort_values(by='date')

    charts['timeline'] = px.line(df_daily, x='date', y='amount',
                                 title='Daily Spending')
    display(charts['timeline'])


In [38]:
if 'date' in df.columns:    
    # Calculate cumulative sum
    df_daily['cumulative'] = df_daily['amount'].cumsum()

    charts['timeline_accumulated'] = px.line(df_daily, x='date', 
                                            y='cumulative',
                                            color_discrete_sequence=['red'],
                                            title='Daily Spending (accumulated)')
    display(charts['timeline_accumulated'])

In [ ]:
# both lines

if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])  # Ensure datetime

    # only use expenses!
    df_expense = df[df['transaction_type'] == 'Expense'].copy()
    df_expense['amount'] = df_expense['amount'].abs()

    # Group by date and sum amounts (daily spending)
    df_daily = df_expense.groupby('date', as_index=False)['amount'].sum()
    df_daily = df_daily.sort_values(by='date')

    # Calculate cumulative sum
    df_daily['cumulative'] = df_daily['amount'].cumsum()

    # Reshapes to long format for px.line (see cell below!)
    df_long = df_daily.melt(id_vars='date', # used as index
                            value_vars=['amount', 'cumulative'], # combine these two columns into a single column
                            var_name='Type', # will indicate the type of value (daily or cumulative)
                            value_name='Value') # will hold the values from amount and cumulative

    fig = px.line(
        df_long,
        x='date',
        y='Value',  
        color='Type',
        title='Daily and Cumulative Spending'
    )
    fig.show()

In [29]:
display(df_long)

,date,Type,Value
0,2025-01-01,amount,120.91
1,2025-01-03,amount,27.56
2,2025-01-06,amount,14.38
3,2025-01-08,amount,202.40
4,2025-01-09,amount,85.78
5,2025-01-10,amount,6.82
6,2025-01-11,amount,34.00
7,2025-01-13,amount,96.71
8,2025-01-14,amount,63.44
9,2025-01-15,amount,16.16
